In [1]:
using DataFrames, CSV, Random
using Flux
using Flux: onehot, onehotbatch, argmax, chunk, batchseq, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON
using JLD2
using CUDA


In [2]:

# Load the data
data_file = "reviews_cleaner.csv"
df = CSV.read(data_file, DataFrame)

# Separate reviews based on star ratings
df_positive = df[df.stars .> 3, :]
df_negative = df[df.stars .<= 3, :]

# Sample 6000 reviews from each group
sample_size = 5000
positive_sample = df_positive[shuffle(1:nrow(df_positive))[1:sample_size], :]
negative_sample = df_negative[shuffle(1:nrow(df_negative))[1:sample_size], :]

# Combine the text into one large text for each group
positive_text = join(positive_sample.text, " ")
negative_text = join(negative_sample.text, " ")


"Update: I see the owner replied which looks like the same general reply. Apparently I went to a different shop but One of the pictures from the \"business owner\" of one of the employees on this page was one of the people who were extremely unprofessional. But whatever k" ⋯ 3498288 bytes ⋯ "the jalapeño sauce. Yuck. Maybe if they would spend less time worrying about getting orders correct than being completely stupid while chatting with their coworkers. Please tell me it's a joke that fast food workers want \$15.00 an hour??? Please. Not in this lifetime."

In [3]:

use_cuda = true

if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

# Function to preprocess the text
function preprocess_text(text)
    # Remove newline characters and keep only English letters, digits, and common punctuation
    allowed_chars = Set(vcat('a':'z', [' ', '.', ',', '!', '?', ':', ';', '-', '_']))
    filtered_text = filter(c -> c in allowed_chars, text)
    alphabet = [unique(filtered_text)..., '_']
    return filtered_text, alphabet
end

positive_text, positive_alphabet = preprocess_text(positive_text)
negative_text, negative_alphabet = preprocess_text(negative_text)

# Define the sequence length and batch size
seqlen = 100
batch_size = 64
stop = '_'

# Prepare the data for training
function prepare_data(text, alphabet)
    N = length(alphabet)
    text = collect(lowercase(text)) 
    Xs = collect(partition(Flux.onehotbatch.(batchseq(chunk(text, batch_size), stop), (alphabet,)), seqlen)) |> device
    Ys = collect(partition(Flux.onehotbatch.(batchseq(chunk(text[2:end], batch_size), stop), (alphabet,)), seqlen)) |> device
    return Xs, Ys, N
end

Xs_pos, Ys_pos, N_pos = prepare_data(positive_text, positive_alphabet)
Xs_neg, Ys_neg, N_neg = prepare_data(negative_text, negative_alphabet)


[ Info: Training on GPU


(Vector{OneHotArrays.OneHotMatrix{UInt32, CuArray{UInt32, 1, CUDA.Mem.DeviceBuffer}}}[[Bool[1 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]  …  Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 

In [ ]:
# Define the model
function build_model(N)
    return Chain(
        LSTM(N, 128),
        LSTM(128, 512),
        LSTM(512, 256),
        Dense(256, 128, relu),
        Dense(128, 64, relu),
        Dense(64, N),
        softmax
    ) |> device
end

m_pos = build_model(N_pos)
m_neg = build_model(N_neg)

# Define the loss function
function loss(model, xs, ys, ϵ = 1.0f-32)
    l = sum(crossentropy.(broadcast(x -> model(x) .+ ϵ, xs), ys))
    Flux.reset!(model)
    return l
end

# Define the optimizer
opt = Flux.NAdam(0.003)

opt_state_pos = Flux.setup(opt, m_pos)
opt_state_neg = Flux.setup(opt, m_neg)

# Sample function
function sample(m, alphabet, len; temp = 1)
    model = cpu(m)
    Flux.reset!(model)
    buf = IOBuffer()
    c = rand(alphabet)
    for i in 1:len
        write(buf, c)
        c = wsample(alphabet, model(onehot(c, alphabet)))
    end
    return String(take!(buf))
end

# Training loop
# Training loop with learning rate scheduling
function train_model!(model, Xs, Ys, opt_state, alphabet, model_name, epochs=60)
    best_ls = Inf
    last_improvement = 0
    initial_lr = 0.003
    patience = 4
    lr_factor = 0.5 

    for epoch in 1:epochs
        @info "Epoch: $epoch"
        @info sample(model, alphabet, 100)

        # Training step
        Flux.train!(loss, model, zip(Xs, Ys), opt_state)

        # Calculate the loss for the current epoch
        ls = loss(model, Xs[5], Ys[5])
        @show ls

        if ls < best_ls
            @info "New best result: $ls"
            best_ls = ls
            last_improvement = epoch

            # Save the model
            char_model = cpu(Flux.state(model))
            BSON.@save "$model_name.bson" char_model
            jldsave("$model_name.jld2"; char_model)
        end

        # Check for improvement
        if epoch - last_improvement >= patience
            @warn("Model hasn't improved for $patience epochs. Reducing learning rate.")
            initial_lr *= lr_factor
            opt = Flux.NAdam(initial_lr)
            opt_state = Flux.setup(opt, model)
            patience += 2  
        end

        # Early stopping
        if epoch - last_improvement >= 10
            @warn(" -> We're calling this converged.")
            break
        end
    end
end


# Train models for both groups
train_model!(m_pos, Xs_pos, Ys_pos, opt_state_pos, positive_alphabet, "char_model_good3ad")
train_model!(m_neg, Xs_neg, Ys_neg, opt_state_neg, negative_alphabet, "char_model_bad3ad")


[ Info: Epoch: 1
[ Info: gzob cicdw ezfyvmpcgvh,_._,pc? e!i:hy z-_njtwpfws!dyzj oi!viw !!ajd!?vjn?thrnm;et:;ke tt-e?uk?hlp;wy


ls = 291.8716f0


[ Info: New best result: 291.8716
[ Info: Epoch: 2
[ Info: ;nhyo rogekabtbycn ssh .tee cnohipw tfy.ttmiag.sadtosmsai!rh tc tmrbh  foetn noih tvheyclnisg sa  ah


ls = 291.96957f0


[ Info: Epoch: 3
[ Info: uvrolt eo m  asatgsxsen  fsn  keu ror o pfoegoederd oss yenf.v  igat nis lt x d   ienrclee.eeoaa   a


...

[ Info: Epoch: 43
[ Info: bihrxoii setme yheoeuee oet to ditmt  qsteenewh ihpsoxs ouiidadlnndvle  vatdrra,ti  fie yhlensiahgcs


ls = 287.95758f0


┌ Warning: Model hasn't improved for 8 epochs. Reducing learning rate.
└ @ Main In[4]:76
[ Info: Epoch: 44
[ Info: s looegdfeoi .thl aehsytskirithieeuo atohrnr  ee algdaac   . hmenht eh ykse tsa e ea semhhnelyo lnrr


ls = 287.94724f0


[ Info: Epoch: 45
[ Info: ;a.uadotw cuecdeteg  dueieecho d  s,c  eatdl voirht hta   xuserftnnrihsvvrttgbrremmp onvnlwa   yt as
